In [19]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import pandas as pd
import requests
from selenium import webdriver

In [20]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [21]:
# Visit NASA news
url = "https://mars.nasa.gov/news"
browser.visit(url)

time.sleep(1)

In [22]:
# Scrape the page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [29]:
# Get the latest news title (inspected the webpage)
result = soup.find_all('div', class_='content_title')
news_title = result[1].text
news_title

'Tricky Terrain: Helping to Assure a Safe Rover Landing'

In [32]:
# Get news para
news_para = soup.find('div', class_='article_teaser_body').text
# news_para


'How two new technologies will help Perseverance, NASA’s most sophisticated rover yet, touch down onto the surface of Mars this month.'

In [21]:
# Visit the JPL url to get the image
image_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg"
browser.visit(image_url)

time.sleep(1)

In [22]:
# Scrape the image into Soup
img_html = browser.html
img_soup = bs(img_html, 'html.parser')

In [30]:
# Get the image url
for link in img_soup.find_all('img'):
    featured_image_url = link.get('src')

In [31]:
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


In [32]:
# Set the url
marsfacts = 'https://space-facts.com/mars/'

In [33]:
# use read_html to scrape out tables and select the relevant table
facts_table = pd.read_html(marsfacts)
df = facts_table[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [34]:
# Rename columns
df.columns = ['Facts', 'Value']
df.set_index('Facts', inplace=True)
df

,Value
Facts,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [35]:
# Convert the dataframe to an HTML table string
facts_html = df.to_html()
facts_html = facts_html.replace('\n', '')
facts_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Facts</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [36]:
# Visit the Mars Hemispheres site
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url)

time.sleep(1)

In [37]:
# Scrape page into Soup
usgs_html = browser.html
usgs_soup = bs(usgs_html, 'html.parser')

In [39]:
# Inspect and find relevant tags
usgs_all = usgs_soup.find_all('div', class_='item')
usgs = 'https://astrogeology.usgs.gov'

In [41]:
# Define an empty list to hold the urls
hemisphere_urls = []

# Run a loop 
for x in usgs_all:
    # Get the title of the hemisphere image
    title = x.find('h3').text
            
    # Get the path of hemisphere image
    img_path = x.find('a',class_='itemLink product-item')['href']
            
    #Visit the image url
    browser.visit(usgs + img_path)
            
    # Scrape the page
    usgs_html_path = browser.html
    usgs_soup_html = bs(usgs_html_path, 'html.parser')
            
    # Get the full image url
    img_tag = usgs_soup_html.find('div', class_ = 'downloads')
    hem_img_url = img_tag.find('a')['href']
            
    #Append the dictionary with the image url string and the hemisphere title
    hemisphere_urls.append({'title': title, 'img_url': hem_img_url})
        
# Display the Mars Hemisphere image url list
print(hemisphere_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
